In [1]:
import numpy as np

In [2]:
arr = np.load('in.npy')

In [3]:
numpy_array = arr.transpose(0,1,4,2,3)


In [17]:
out = np.load('out.npy')
out = out.transpose(0,3,1,2)

In [18]:
out.shape

(8000, 1, 512, 512)

In [4]:
import os
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm   
import torch.nn.functional as F

from models.PConv_model import PConvUNet
from data_prep import Data_prep
from net import VGG16FeatureExtractor
from loss import InpaintingLoss, IOA_Loss
from torch.utils.data import TensorDataset, DataLoader, random_split
from tensorboardX import SummaryWriter
from torch.utils import data
import yaml

from util.io import load_ckpt
from util.io import save_ckpt
from torchvision.utils import make_grid
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [5]:
numpy_array.shape

(2, 8000, 3, 512, 512)

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class ImageMaskDataset(Dataset):
    def __init__(self, data_array, out, transform=None):
        self.data_array = data_array
        self.transform = transform
        self.out = out

    def __len__(self):
        return self.data_array.shape[1]

    def __getitem__(self, idx):
        image = self.data_array[0, idx]
        mask = self.data_array[1, idx]
        out = self.out[idx]
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        return image, mask, out

from torch.utils.data.sampler import Sampler

class InfiniteSampler(Sampler):
    def __init__(self, num_samples):
        self.num_samples = num_samples

    def __iter__(self):
        return iter(self.loop())

    def loop(self):
        i = 0
        order = np.random.permutation(self.num_samples)
        while True:
            yield order[i]
            i += 1
            if i >= self.num_samples:
                np.random.seed()
                order = np.random.permutation(self.num_samples)
                i = 0


In [27]:


# Assuming `main_array` is your input array
dataset = ImageMaskDataset(data_array=numpy_array,out=out)

dataloader = DataLoader(
    dataset, 
    batch_size=4, 
    sampler=InfiniteSampler(len(dataset)), 
    num_workers=4  # Adjust as per your computational resources
)


In [28]:

dataloader

In [29]:
img = next(iter(dataloader))

In [30]:
img[0].shape, img[1].shape

(torch.Size([4, 3, 512, 512]), torch.Size([4, 3, 512, 512]))

In [32]:
img[2].shape

torch.Size([4, 1, 512, 512])

In [36]:
full_dataset = ImageMaskDataset(data_array=numpy_array, out=out)
train_size = int(0.8 * len(full_dataset))
eval_size = len(full_dataset) - train_size
train_dataset, eval_dataset = random_split(full_dataset, [train_size, eval_size])
        
train_dataloader = DataLoader(
    train_dataset, 
    4, 
    sampler=InfiniteSampler(len(train_dataset)), 
    num_workers=10  
)


# Creating the eval DataLoader
eval_dataloader = DataLoader(
    eval_dataset, batch_size=4,
    # No need for the InfiniteSampler for evaluation
    shuffle=False,
    num_workers=10)

In [38]:
len(train_dataloader)

TypeError: object of type 'InfiniteSampler' has no len()